In [1]:
import numpy as np

1. Defining the weight matrix, where each of the first columns are biases=1.
2. Input matrix is also appended by 1 at the start to show the bias.


In [2]:
inp = np.asarray([1, 1, 1, 0, 0, 1, 1], dtype=np.float32)
alpha = np.asarray([[1, 1, 2, -3, 0, 1, -3],
                    [1, 3, 1, 2, 1, 0, 2],
                    [1, 2, 2, 2, 2, 2, 1],
                    [1, 1, 0, 2, 1, -2, 2]], dtype = np.float32)
beta = np.asarray([[1, 1, 2, -2, 1],
                   [1, 1, -1, 1, 2],
                   [1, 3, 1, -1, 1]], dtype = np.float32)


In [3]:
a = alpha @ inp.T
a

array([2., 7., 8., 2.], dtype=float32)

In [4]:
z = 1. / (1 + np.exp(-a))
z
z = np.insert(z, 0, 1)
z

array([1.        , 0.880797  , 0.999089  , 0.99966466, 0.880797  ],
      dtype=float32)

In [5]:
b = beta @ z.T
b

array([2.7604427, 3.6429667, 4.5226126], dtype=float32)

In [6]:
y = np.exp(b) / np.sum(np.exp(b))
y

array([0.10820103, 0.26152113, 0.6302779 ], dtype=float32)

1. From the above arrays we can see that the predicted class is 3.
2. The answers are reported separately

In [7]:
loss = - np.log(y[1])
loss

1.3412402

\begin{equation}
\frac{\partial loss}{\partial y} * \frac{\partial y}{\partial b} * \frac{\partial b}{\partial \beta} = \frac{\partial loss}{\partial \beta}
\end{equation} \
db_dbeta represents the derivative of b w.r.t beta matrix. For the sake of matrix manipulation it is repeated 3 times to make 3 rows \
dy_db represensts derivative of y w.r.t to b, which is the derivative of the softmax \
dloss_dy represents derivative of total loss w.r.t to y, Only 1 derivative is non-zero rest are zero.

In [8]:
db_dbeta = np.asarray([[1,0.880797,.990889,.999665,.880797],
                       [1,0.880797,.990889,.999665,.880797],
                       [1,0.880797,.990889,.999665,.880797]])

dy_db = np.asarray([[-.028297],
                [.193128],
                [-.164831]])

dloss_dy = np.asarray([[-3.823785]])

dloss_dbeta = db_dbeta * dy_db * dloss_dy
dloss_dbeta

array([[ 0.10820164,  0.09530368,  0.10721582,  0.1081654 ,  0.09530368],
       [-0.73847995, -0.65045092, -0.73175166, -0.73823256, -0.65045092],
       [ 0.63027831,  0.55514724,  0.62453584,  0.63006716,  0.55514724]])

dloss_dbeta represents: ∂𝑙𝑜𝑠𝑠/∂𝛽 \
by gradient descent we change 𝛽 -= ∂𝑙𝑜𝑠𝑠/∂𝛽 * lr \
lr = 1

In [9]:
beta -= dloss_dbeta 
beta

array([[ 0.8917984 ,  0.90469635,  1.8927842 , -2.1081655 ,  0.90469635],
       [ 1.73848   ,  1.650451  , -0.26824835,  1.7382326 ,  2.650451  ],
       [ 0.36972168,  2.4448528 ,  0.37546417, -1.6300671 ,  0.44485277]],
      dtype=float32)

To find derivatives w.r.t alpha \
\begin{equation}
\frac{\partial loss}{\partial y} * \frac{\partial y}{\partial b} * \frac{\partial b}{\partial z} * \frac{\partial z}{\partial a} * \frac{\partial a}{\partial \alpha} = \frac{\partial loss}{\partial \alpha}
\end{equation} \
calculates this value (∂y/∂b)∗(∂b/∂z) for z1, z2, z3, z4 by using appropriate b's, and is made into the matrix dy_db_dz

In [10]:
dy_db_dz1 = np.asarray([-y[0]* y[1]*1, y[1]*(1-y[1])*1, -y[1]*y[2]*3])
dy_db_dz2 = np.asarray([-y[0]* y[1]*2, y[1]*(1-y[1])*-1, -y[1]*y[2]*1])
dy_db_dz3 = np.asarray([-y[0]* y[1]*-2, y[1]*(1-y[1])*1, -y[1]*y[2]*-1])
dy_db_dz4 = np.asarray([-y[0]* y[1]*1, y[1]*(1-y[1])*2, -y[1]*y[2]*1])
dy_db_dz = np.asarray([[np.sum(dy_db_dz1)], [np.sum(dy_db_dz2)], [np.sum(dy_db_dz3)], [np.sum(dy_db_dz4)]]) 
dy_db_dz 

array([[-0.32966198],
       [-0.41455252],
       [ 0.41455252],
       [ 0.19312782]])

Calculating (∂z/∂a), which is the derivative of sigmoid for z1, z2, z3, z4 wrt corresponding a's and is made into a 4*1, matrix dz_da

In [11]:
dz_da = np.asarray([[z[1]*(1-z[1])], [z[2]*(1-z[2])], [z[3]*(1-z[3])], [z[4]*(1-z[4])]])
dz_da 

array([[0.10499362],
       [0.00091017],
       [0.00033522],
       [0.10499362]])

To get (∂a/∂alpha) the input matrix is repeated 4 times. And all the derivative matrices are multiplied

In [12]:
da_dalpha = np.asarray([[1, 1, 1, 0, 0, 1, 1],
                        [1, 1, 1, 0, 0, 1, 1],
                        [1, 1, 1, 0, 0, 1, 1],
                        [1, 1, 1, 0, 0, 1, 1]], dtype = np.float32)

dloss_dalpha = (dy_db_dz * dz_da) * da_dalpha * dloss_dy
dloss_dalpha

array([[ 0.1323504 ,  0.1323504 ,  0.1323504 ,  0.        ,  0.        ,
         0.1323504 ,  0.1323504 ],
       [ 0.00144276,  0.00144276,  0.00144276,  0.        ,  0.        ,
         0.00144276,  0.00144276],
       [-0.00053138, -0.00053138, -0.00053138, -0.        , -0.        ,
        -0.00053138, -0.00053138],
       [-0.07753561, -0.07753561, -0.07753561, -0.        , -0.        ,
        -0.07753561, -0.07753561]])

Doing the gradient descent on alpha and updating

In [13]:
alpha -= dloss_dalpha 
alpha

array([[ 8.6764961e-01,  8.6764961e-01,  1.8676496e+00, -3.0000000e+00,
         0.0000000e+00,  8.6764961e-01, -3.1323504e+00],
       [ 9.9855721e-01,  2.9985573e+00,  9.9855721e-01,  2.0000000e+00,
         1.0000000e+00, -1.4427608e-03,  1.9985572e+00],
       [ 1.0005314e+00,  2.0005314e+00,  2.0005314e+00,  2.0000000e+00,
         2.0000000e+00,  2.0005314e+00,  1.0005314e+00],
       [ 1.0775356e+00,  1.0775356e+00,  7.7535614e-02,  2.0000000e+00,
         1.0000000e+00, -1.9224644e+00,  2.0775356e+00]], dtype=float32)

Final Prediction which is 2.

In [14]:
a = (alpha @ inp.T)
z = a / (1 + np.exp(-a))
z = np.insert(z, 0, 1)
b = beta @ z.T
y = np.exp(b) / np.sum(np.exp(b))
y

array([6.669451e-10, 1.000000e+00, 8.454082e-13], dtype=float32)